# A Reporting Case-Control Study of AEFIs Following the COVID-19 Vaccine Reported to VAERS

_Chris von Csefalvay_

_04 July 2021_

## Analysis plan

1. Import VAERS data from 2015 onwards, to give a sufficient number of records.
2. We filter out unknown ages and genders.
3. We then bracket age groups: <18, 18-25, 26-35, 36-45, 46-55, 56-65, 66-75, 76-85, 85+
4. We use `pymatch` to create a matched set of controls by:
    * age band
    * gender
5. We drop all events that have fewer occurrences than 0.5% of the higher of the two sets' cardinalities.
6. We filter out all diagnostic, procedural &c. codes.
7. Finally, we calculate the reporting odds ratios of all symptoms.

In [27]:
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pymatch
from pymatch import Matcher
import sys, re, os, glob
from tqdm.autonotebook import tqdm

tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"statsmodels version: {statsmodels.__version__}")

pandas version: 1.3.0
numpy version: 1.21.0
statsmodels version: 0.12.2


## Analysis

### Loading VAERS data

Note: for legal reasons, we cannot reproduce the raw VAERS data in Github. However, you can download the VAERS data set
from [HHS](https://vaers.hhs.gov).

In [28]:
path = "../data/" # use your path
vaccine_files = glob.glob(path + "*VAERSVAX.csv")

vax_frames = []

for filename in tqdm(vaccine_files):
    df = pd.read_csv(filename, index_col=None, header=0, encoding="latin")
    vax_frames.append(df)

vax = pd.concat(vax_frames, axis=0, ignore_index=True)[["VAERS_ID", "VAX_TYPE"]]
vax["VAX_TYPE"] = vax["VAX_TYPE"] == "COVID19"
vax.columns = ["VAERS_ID", "IS_COVID_VACCINE"]

  0%|          | 0/27 [00:00<?, ?it/s]

In [29]:
recipient_files = glob.glob(path + "*VAERSDATA.csv")

recipient_frames = []

for filename in tqdm(recipient_files):
    df = pd.read_csv(filename, index_col=None, header=0, encoding="latin")
    recipient_frames.append(df)

recipients = pd.concat(recipient_frames, axis=0, ignore_index=True)[["VAERS_ID", "SEX", "CAGE_YR"]]

  0%|          | 0/27 [00:00<?, ?it/s]

In [30]:
symptoms_files = glob.glob(path + "*VAERSSYMPTOMS.csv")

symptoms_frames = []

for filename in tqdm(symptoms_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    symptoms_frames.append(df)


symptoms = pd.melt(pd.concat(symptoms_frames, axis=0, ignore_index=True)[["VAERS_ID", "SYMPTOM1", "SYMPTOM2", "SYMPTOM3", "SYMPTOM4", "SYMPTOM5"]],
               id_vars="VAERS_ID",
               value_vars=(f"SYMPTOM{i}" for i in range(1, 6))).drop("variable", axis=1)

symptoms.columns = ("VAERS_ID", "SYMPTOM")

  0%|          | 0/27 [00:00<?, ?it/s]

In [31]:
vaccination_data = vax.merge(recipients, how="inner", on="VAERS_ID")

In [36]:
print(f"Retrieved {vaccination_data.shape[0]} rows.")
print(f"Breakdown:")
print(f"* COVID-19: {vax[vax.IS_COVID_VACCINE == True][['VAERS_ID']].count().values[0]}")
print(f"* Non-COVID-19: {vax[vax.IS_COVID_VACCINE == False][['VAERS_ID']].count().values[0]}")
print(f" => Gearing: {vax[vax.IS_COVID_VACCINE == False][['VAERS_ID']].count().values[0]/vax[vax.IS_COVID_VACCINE == True][['VAERS_ID']].count().values[0]:.2f}")

Retrieved 1421356 rows.
Breakdown:
* COVID-19: 417186
* Non-COVID-19: 1004170
 => Gearing: 2.41
